# Tâche #1 : Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On reprend la classification des descriptions d’accidents du premier travail. Le corpus de textes contient 3 partitions : 
-	un fichier d’entraînement -  data/incidents_train.json
-	un fichier de validation -  data/incidents_dev.json
-	un fichier de test - data/incidents_test.json

Entraînez un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

Voici les consignes pour cette tâche : 

-	Nom du notebook : mlp.ipynb
-	Tokenisation : Utilisation de Spacy. 
-	Plongements de mots : Ceux de Spacy. 
-	Normalisation : Aucune normalisation. 
-	Agrégation des plongements de mots : Comparer les approches max, average et min pooling. 
-	Structure du réseau : 1 seule couche cachée dont vous choisirez la taille (à expliquer). 
-	Présentez clairement vos résultats et faites-en l’analyse. En cas de doute, inspirez-vous de ce qui a été fait dans le travail pratique #1. 

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [1]:
print('a')

a


## 2. Gestion de plongements de mots (*embeddings*)

## 3. Création de modèle(s)

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

## 6. Évaluation et analyse de résultats